In [150]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from urllib import parse
import pytesseract
import numpy as np
import time
import re
import cv2
import json

def url_to_image(url):
    try:
        res = urllib.request.urlopen(url)
        image = np.asarray(bytearray(res.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    except:
        image = cv2.imread('sample.jpg')
    return image

def ocr(image):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.bitwise_not(cv2.adaptiveThreshold(img_gray, 255, cv2.THRESH_BINARY, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 75, 10))
    text = pytesseract.image_to_string(img_gray, lang="kor")
    return text

def get_blog_url(query,start_date,stop_date):
    browser="C:\\Users\CPB06GameN\Anaconda3\envs\chromedriver.exe"
    driver=webdriver.Chrome(browser)

#     query = '다이슨'
#     start_date = '20190902'
#     stop_date = '20190908'
    url = 'https://search.naver.com/search.naver?where=post&query='+query+'&date_from='+start_date+'&date_to='+stop_date+'&date_option=8&post_blogurl=naver.com'
    driver.get(url)

    total = driver.find_element_by_css_selector('#main_pack > div.blog.section._blogBase._prs_blg > div > span').text
    total = total.split(' ')
    for i in range(0,len(total)):
        if len(re.findall('.*건',total[i])) > 0:
            tot = total[i]
    tot = tot.replace(',','').replace('건','')
    page = int(tot)//10
    if page > 100:
        page = 100
    url_list = []

    for i in range(0,page):
        for j in range(1,11):
            blog_url = driver.find_element_by_css_selector('#sp_blog_'+str(j)+' > dl > dt > a').get_attribute('href')
            url_list.append(blog_url)
        try:
            driver.find_element_by_css_selector('#main_pack > div.paging > a.next').click() 
            time.sleep(2)
        except:
            pass
    return url_list

def blog_info(url):
    browser="C:\\Users\CPB06GameN\Anaconda3\envs\chromedriver.exe"
    driver=webdriver.Chrome(browser)
    driver.get(url)
    driver.switch_to.frame('mainFrame')
    html = driver.page_source
    
    soup = BeautifulSoup(html,'html.parser')
    text_list = soup.find_all('div',{'class' : 'se-module se-module-text'})
    fin = ''
    last_text = ''
    img_url = ''
    title = ''
    date = ''
    for i in text_list:
        try:
            txt = i.find_all('p',{'class':'se-text-paragraph se-text-paragraph-align-center'})
        except:
            pass
        for j in txt:
            fin = fin + j.text
            last_text = j.text
    fin = fin.replace('\u200b','')
    try:
        date = soup.find('span',{'class': 'se_publishDate pcol2'}).text
    except:
        pass
    try:
        title = soup.find('div',{'class': 'se-module se-module-text se-title-text'}).text.replace('\n','')
    except:
        pass
    
    try:
        img_url = soup.find_all('div',{'class': 'se-component se-image se-l-default'})[-1].find('img')['src']
    except:
        pass
    
    korean = re.findall('[ㄱ-ㅣ가-힣]+',img_url)
    for i in range(0,len(korean)):
        parse.quote_plus(korean[i])
        img_url = img_url.replace(korean[i],parse.quote(korean[i]))
   
    driver.close()
    
    return title, date, fin, img_url, last_text


def make_json(data,filename):
    with open(filename+'.json','w',encoding='utf-8') as make_file:
        json.dump(data, make_file, ensure_ascii=False, indent='\t')

In [81]:
list_url = get_blog_url('다이슨','20190902','20190908')

In [151]:
json_list=[]
for i in range(0,30):
    title, date, text, img_url, last_text = blog_info(list_url[i])
    img = url_to_image(img_url)
    ocr_txt = ocr(img)
    blog_data = {}
    p = re.compile('[제,받,고]+[공,아,료]')
    if p.search(ocr_txt):
        print(ocr_txt)
    elif p.search(last_text):
        print(last_text)
    else:
        blog_data['title'] = title
        blog_data['date'] = date
        blog_data['text'] = text
        json_list.append(blog_data)
        
make_json(json_list,'광고아닌블로그')


<<본포스팀은 다이슨으로부터 제품을 재공받아 실재 체험을 통해 작성되었습니다.>
제품 소개를 대가로 다이슨으로부터 제품 무상 대여와 원고료를 지원받음
0“ | 글과 사진, 이퓨림

블로그의 모든 사진과 글은
무단도용 / 2차 수정 / 불펌을 금지합니다.          )
ㆍ이 포스팅은 다이슨으로부터 제품을 제공받아 실제 체험을 통해 작성되었습니다.      2
미 포스팅몬 다이슨으로부터 소정의 고료를 받고
직접 체험을 통해 작성된 포스팅입니다.
「   9   %

~
2년 무상 보증      대여 제품 무상 제공     72시간이내 수리       제품 기술 지원
{사용자 설명서 참조} 수리기간 동안 불편함을 최소화하기 택배 및 방문 서비스롤 통해제품을 제풍에 대한 문의사항은 평일(월~금)
플                   위해 대여 제품을 무상으로 임대해 수거하여 72시간 내 수리를           오전 9시~오후 6시 사이에 전화,
보중 기간 이내 수리가 불가능할                 드립니다.                          완료합니다.                  이메일, 채팀을 통해 확인하실 수
우 신속하게 교체해 드립니다  ,입반 택비 배송기간 소요,    (제품 배송 및 픽업 시간 제외)         있습니다.

(소비자분정해결기준 적용)             여비 45 점수 걷에 한함)
* 본 폰스팅은 소정의 원고로를 지급받아 작석되었습닝다."


error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
